In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import model_selection, metrics
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras
from time import time
import glob
from keras.callbacks import TensorBoard


import os
print(os.listdir("."))

['.ipynb_checkpoints', 'model2.csv', 'test.csv', 'train.csv', 'Untitled1.ipynb', 'model0.csv', 'model3.ipynb', 'model2.ipynb', 'model1.csv', 'model1.ipynb', 'mnist_submit.ipynb', 'EDA.ipynb', 'Untitled.ipynb']


In [42]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [43]:
def load_data(train_path='train.csv', test_path='test.csv'):
    train = pd.DataFrame.from_csv(train_path, index_col=None)
    test = pd.DataFrame.from_csv(test_path, index_col=None)
              
    return train, test

In [44]:
train, test = load_data()

/home/lazukav/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/home/lazukav/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
train_X = train.drop("label", axis=1)
train_y = train["label"]

In [46]:
train_X = train_X.div(255, axis=0).fillna(0)
test = test.div(255, axis=0).fillna(0)

In [47]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_X, train_y, test_size=0.2, shuffle=None)

In [48]:
cols = [tf.feature_column.numeric_column(i) for i in train_X.columns.values]

In [49]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [50]:
X_train = X_train.values.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)

In [51]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [52]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [53]:
batch_size = 128
num_classes = 10
epochs = 5 # 12
pool_size = 2
kernel_size = 3

In [54]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Activation, Dropout, Flatten

In [55]:
x_in = Input(shape=X_test.shape[1:])
x = Conv2D(32, (3, 3),
                  activation='relu', padding='valid')(x_in)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes,
                           activation='softmax',
                           name='x_train_out')(x)

model3 = Model(inputs=x_in, outputs=predictions)
model3.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model3.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[TrainValTensorBoard(write_graph=False,log_dir="/tmp/model3")])
score = model3.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 33600 samples, validate on 8400 samples
Epoch 1/5
33600/33600 [==============================] - 14s 405us/step - loss: 0.3540 - acc: 0.8880 - val_loss: 0.1324 - val_acc: 0.9575
Epoch 2/5
33600/33600 [==============================] - 13s 399us/step - loss: 0.0890 - acc: 0.9719 - val_loss: 0.0657 - val_acc: 0.9796
Epoch 3/5
33600/33600 [==============================] - 13s 398us/step - loss: 0.0626 - acc: 0.9801 - val_loss: 0.0519 - val_acc: 0.9848
Epoch 4/5
33600/33600 [==============================] - 13s 398us/step - loss: 0.0476 - acc: 0.9853 - val_loss: 0.0555 - val_acc: 0.9826
Epoch 5/5
33600/33600 [==============================] - 13s 397us/step - loss: 0.0394 - acc: 0.9872 - val_loss: 0.0544 - val_acc: 0.9843
Test loss: 0.054399716141411945
Test accuracy: 0.9842857142857143


In [56]:
scores3 = model3.predict(test.values.reshape(test.shape[0], 28, 28, 1))

res = []
for i, p in enumerate(scores3.argmax(axis=-1)):
    res.append([i + 1, p])   

result = pd.DataFrame(res, columns=['ImageId', 'Label'])
result.to_csv('model3.csv', index=None)